# NN with transfer learning

This is implemented on workspace with CUDA suppport

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets, transforms, models
import time

In [3]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = 'Cat_Dog_data'  # workspace folder
# data_dir = '~/.pytorch/Cat_Dog_data/Cat_Dog_data' # local folder

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [4]:

# load the pretrained model - Densenet

# model = models.densenet121(pretrained=True)
model = models.resnet50(pretrained=True)

# freeze the parameters to prevent bachpropagation on features
for param in model.parameters():
    param.requires_grad = False
    
# create your classifier
classifier = nn.Sequential(nn.Linear(2048, 512),
                          nn.ReLU(),
                          nn.Dropout(p=0.2),
                          nn.Linear(512, 2),
                          nn.LogSoftmax(dim=1))
# add classifier to model
model.fc = classifier
# print(model.fc)

# Definitions

criterion = nn.NLLLoss()

# optimize only classifier parameters
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# move the model to available device
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:02<00:00, 41807773.61it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [ ]:
# Training 
epochs = 1
running_loss = 0
print_every = 5
steps = 0

for epoch in range(epochs):
    for images, labels in trainloader:        
        steps +=1
        
        # Move input tensors to device
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # validate the model for every few training batches 
        if steps % print_every == 0:
            
            #turn off dropouts
            model.eval()
            test_loss = 0
            accuracy = 0
            
            with torch.no_grad():
                for images, labels in testloader:
                    # Transfer the tensors to gpu
                    images, labels = images.to(device), labels.to(device)  

                    logps = model(images)
                    loss = criterion(logps, labels)                
                    test_loss += loss.item()

                    # Calculate accuracy                    
                    ps = torch.exp(logps)
                    top_prob, top_class =  ps.topk(1, dim=1)
                    equality = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equality.type(torch.FloatTensor))
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            
            # switch back to training
            model.train()
        

Epoch 1/1.. Train loss: 0.125.. Test loss: 0.100.. Test accuracy: 0.966
Epoch 1/1.. Train loss: 0.245.. Test loss: 0.102.. Test accuracy: 0.966
Epoch 1/1.. Train loss: 0.289.. Test loss: 0.092.. Test accuracy: 0.965
Epoch 1/1.. Train loss: 0.257.. Test loss: 0.058.. Test accuracy: 0.979
Epoch 1/1.. Train loss: 0.215.. Test loss: 0.104.. Test accuracy: 0.961
Epoch 1/1.. Train loss: 0.210.. Test loss: 0.124.. Test accuracy: 0.950
Epoch 1/1.. Train loss: 0.226.. Test loss: 0.058.. Test accuracy: 0.981
Epoch 1/1.. Train loss: 0.168.. Test loss: 0.097.. Test accuracy: 0.963
Epoch 1/1.. Train loss: 0.177.. Test loss: 0.112.. Test accuracy: 0.955
